# How to set a prior given T = 2 non-zero effects?

Assume two non-zero effects $T = 2$. We set PVE $= 0.05$ so that SuSiE has certain power around $0.51$. 
We also set PVE $= 0.7$ so that SuSiE has power around $0.78$.

We investigate SuSiE performance under different priors.

## Results

**- Summary: The choice of prior has a modest influence on SuSiE performance.**

When $pve = 0.05$, 

* a larger prior has a slighler lower FDR and a slightly higher top_hit_rate.

When $pve = 0.7$,

* the choice of prior does not affect SuSiE performance.


In [17]:
singlecell.summary[singlecell.summary$pve==0.05,]

,effect_num,pve,prior,power,fdr,cs_size,cs_num,top_hit_rate,avg_purity
20,2,0.05,0.01,0.49,0.0392,1.021277,1.085106,0.9608,0.9977918
86,2,0.05,0.02,0.51,0.0377,1.021277,1.127660,0.9623,0.9977918
152,2,0.05,0.03,0.51,0.0377,1.021277,1.127660,0.9623,0.9977918
218,2,0.05,0.05,0.53,0.0364,1.021277,1.170213,0.9636,0.9977918
284,2,0.05,0.10,0.53,0.0364,1.021277,1.170213,0.9636,0.9977918
350,2,0.05,0.20,0.53,0.0364,1.021277,1.170213,0.9636,0.9977918
416,2,0.05,0.40,0.53,0.0364,1.021277,1.170213,0.9636,0.9977918
482,2,0.05,0.50,0.53,0.0364,1.021277,1.170213,0.9636,0.9977918
548,2,0.05,0.70,0.53,0.0364,1.021277,1.170213,0.9636,0.9977918
614,2,0.05,0.90,0.53,0.0364,1.021277,1.170213,0.9636,0.9977918


In [16]:
singlecell.summary[singlecell.summary$pve==0.7,]

,effect_num,pve,prior,power,fdr,cs_size,cs_num,top_hit_rate,avg_purity
50,2,0.7,0.01,0.78,0.0127,1.020408,1.612245,0.9747,1
116,2,0.7,0.02,0.78,0.0127,1.020408,1.612245,0.9747,1
182,2,0.7,0.03,0.78,0.0127,1.020408,1.612245,0.9747,1
248,2,0.7,0.05,0.78,0.0127,1.020408,1.612245,0.9747,1
314,2,0.7,0.10,0.78,0.0127,1.020408,1.612245,0.9747,1
380,2,0.7,0.20,0.78,0.0127,1.020408,1.612245,0.9747,1
446,2,0.7,0.40,0.78,0.0127,1.020408,1.612245,0.9747,1
512,2,0.7,0.50,0.78,0.0127,1.020408,1.612245,0.9747,1
578,2,0.7,0.70,0.78,0.0127,1.020408,1.612245,0.9747,1
644,2,0.7,0.90,0.78,0.0127,1.020408,1.612245,0.9747,1


## Code details

In [9]:
singlecell_Q2 = readRDS('singlecell_Q2.rds')
singlecell_Q2 = singlecell_Q2[!is.na(singlecell_Q2$sim_gaussian.output.file),]
singlecell_Q2 = singlecell_Q2[!is.na(singlecell_Q2$susie_prior.output.file),]

In [10]:
singlecell_df = data.frame(singlecell_Q2$sim_gaussian.effect_num, singlecell_Q2$sim_gaussian.pve, singlecell_Q2$susie_prior.prior,
                       singlecell_Q2$score.hit, singlecell_Q2$score.signal_num, singlecell_Q2$score.cs_medianSize,
                       singlecell_Q2$score.top_hit, singlecell_Q2$sim_gaussian.mean_corX, singlecell_Q2$susie_prior.avg_purity)
names(singlecell_df) = c('effect_num', 'pve', 'prior','hit', 'cs_num', 'cs_size', 'top_hit', 'corX', 'avg_purity')

In [11]:
hitsum.summary = aggregate(hit ~ effect_num + pve + prior, singlecell_df, sum)
hitmean.summary = aggregate(hit ~ effect_num + pve + prior, singlecell_df, mean)
power.summary = hitmean.summary
power.summary$power = power.summary$hit / power.summary$effect_num
fdr.summary = aggregate(cs_num ~ effect_num + pve + prior, singlecell_df, sum)
fdr.summary$fdr = round(1 - hitsum.summary$hit / fdr.summary$cs_num, 4)
meannonzero = function(x){mean(x[x!=0])}
cs_num.summary = aggregate(cs_num ~ effect_num + pve + prior, singlecell_df, meannonzero)
setsize.summary = aggregate(cs_size ~ effect_num + pve + prior, singlecell_df, meannonzero)
tophit.summary = aggregate(top_hit ~ effect_num + pve + prior, singlecell_df, sum)
tophit.summary$tophit_rate = round(tophit.summary$top_hit / fdr.summary$cs_num , 4)
singlecell_df$avg_purity[is.na(singlecell_df$avg_purity)]=0
purity.summary = aggregate(avg_purity ~ effect_num + pve + prior, singlecell_df, meannonzero)

In [12]:
singlecell.summary = data.frame(power.summary$effect_num, power.summary$pve, power.summary$prior,
                            power.summary$power, fdr.summary$fdr, 
                            setsize.summary$cs_size, cs_num.summary$cs_num, 
                            tophit.summary$tophit_rate, purity.summary$avg_purity)
names(singlecell.summary) = c('effect_num', 'pve', 'prior','power', 'fdr', 'cs_size', 'cs_num','top_hit_rate', 'avg_purity')

In [13]:
is.nan.data.frame <- function(x)
do.call(cbind, lapply(x, is.nan))
singlecell.summary[is.nan(singlecell.summary)] = 0

In [14]:
singlecell.summary = singlecell.summary[singlecell.summary$effect_num==2, ]